In [1]:
import sbibm
import numpy as np
import torch
import pickle
import pyro
from tqdm import tqdm
from joblib import Parallel, delayed
from pyro.distributions.empirical import Empirical
from functools import reduce
from scipy.optimize import linear_sum_assignment
import timeit

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# sbi
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils
from sbi import analysis

# distances
from scipy.spatial.distance import directed_hausdorff
from torch.nn.functional import normalize

sbibm.get_available_tasks()

['gaussian_mixture',
 'bernoulli_glm',
 'lotka_volterra',
 'two_moons',
 'slcp',
 'gaussian_linear',
 'sir',
 'gaussian_linear_uniform',
 'slcp_distractors',
 'bernoulli_glm_raw']

## Lotka Volterra

### Simulator with summaries

In [2]:
lk = sbibm.get_task("lotka_volterra")  # See sbibm.get_available_tasks() for all tasks
lk_prior = lk.get_prior()
lk_simulator = lk.get_simulator()
lk_observation = lk.get_observation(num_observation=1)  # 10 per task

/home/tt21642/anaconda3/envs/sbi/lib/python3.9/site-packages/diffeqtorch/diffeqtorch.py:29: UserWarning: JULIA_SYSIMAGE_DIFFEQTORCH not set
  warn("JULIA_SYSIMAGE_DIFFEQTORCH not set")
/home/tt21642/anaconda3/envs/sbi/lib/python3.9/site-packages/diffeqtorch/diffeqtorch.py:32: UserWarning: Defaulting to /home/tt21642/.julia_sysimage_diffeqtorch.so
  warn(f"Defaulting to {default_path}")


In [7]:
lk_simulator(lk_prior(num_samples=1))

/home/tt21642/anaconda3/envs/sbi/lib/python3.9/site-packages/diffeqtorch/diffeqtorch.py:255: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/torch/csrc/utils/tensor_new.cpp:210.)
  u = torch.tensor(u).T


tensor([[31.1445, 54.1043,  4.0307,  0.3967,  0.2578,  0.3238,  0.6155,  1.4102,
          3.2500,  8.6279,  1.0144, 19.4609, 87.1144, 43.1918, 13.2248,  6.3567,
          2.7435,  1.3292,  0.6821,  0.4855]])

In [3]:
lk_observation

tensor([[31.7833,  1.1971,  0.2970,  0.7115,  2.9416, 11.4826, 36.0904,  0.4623,
          0.3808,  0.9174,  0.8910, 27.0399,  4.3562,  0.7844,  0.1710,  0.1326,
          8.7270, 16.9725,  2.7902,  0.4757]])

In [17]:
torch.manual_seed(1000020)
lk_simulator(lk_prior(num_samples=1))

tensor([[31.7833,  1.1971,  0.2970,  0.7115,  2.9416, 11.4826, 36.0904,  0.4623,
          0.3808,  0.9174,  0.8910, 27.0399,  4.3562,  0.7844,  0.1710,  0.1326,
          8.7270, 16.9725,  2.7902,  0.4757]])

### Simulator without summaries

In [7]:
lkf = sbibm.get_task("lotka_volterra", summary = None)  # See sbibm.get_available_tasks() for all tasks
lkf_simulator = lkf.get_simulator()

In [11]:
torch.manual_seed(1000020)
lkf_observation=lkf_simulator(lk_prior(num_samples=1))

In [16]:
lkf_observation

tensor([[30.0000, 31.7358, 33.4371, 35.0267, 36.3862, 37.3403, 37.6468, 37.0070,
         35.1293, 31.8668, 27.3902, 22.2446, 17.1615, 12.7362,  9.2377,  6.6478,
          4.8038,  3.5147,  2.6172,  1.9891,  1.5450,  1.2265,  0.9949,  0.8238,
          0.6956,  0.5982,  0.5235,  0.4655,  0.4201,  0.3844,  0.3562,  0.3340,
          0.3166,  0.3031,  0.2928,  0.2853,  0.2802,  0.2771,  0.2758,  0.2762,
          0.2781,  0.2815,  0.2862,  0.2922,  0.2995,  0.3082,  0.3181,  0.3293,
          0.3418,  0.3558,  0.3711,  0.3880,  0.4064,  0.4265,  0.4482,  0.4718,
          0.4974,  0.5249,  0.5547,  0.5867,  0.6212,  0.6584,  0.6983,  0.7412,
          0.7873,  0.8367,  0.8898,  0.9468,  1.0079,  1.0735,  1.1438,  1.2191,
          1.2998,  1.3864,  1.4791,  1.5784,  1.6848,  1.7988,  1.9209,  2.0517,
          2.1918,  2.3418,  2.5024,  2.6743,  2.8585,  3.0556,  3.2666,  3.4926,
          3.7344,  3.9933,  4.2705,  4.5671,  4.8846,  5.2243,  5.5879,  5.9770,
          6.3934,  6.8388,  

In [ ]:
simulator, prior = prepare_for_sbi(rand_simulator, slcp_prior_2)
